In [ ]:
from google.colab import drive

import pandas as pd
import glob
import os


# # Folder path where your CSV files are stored
# folder_path = '/content/nifty'

# # Get a list of all CSV files in the folder
# all_files = glob.glob(os.path.join(folder_path, "*.csv"))

# # Load and concatenate all the CSV files into a single DataFrame
# data = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)

# # Display the first few rows to check the data
# print(data.head())


In [ ]:

folder_path = '/content/nifty'

# List of years in the specific order
years = [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]

# Get a list of all CSV files in the folder
all_files = glob.glob(os.path.join(folder_path, "*.csv"))

# Sort the files based on the order of years in the list
sorted_files = sorted(all_files, key=lambda f: years.index(int(os.path.basename(f).split('.')[0])))

# Load and concatenate the sorted CSV files into a single DataFrame
data = pd.concat((pd.read_csv(f) for f in sorted_files), ignore_index=True)
mata = data.copy()
data.to_csv('data.csv', index=False)

# Display the first few rows to check the data
print(data.head())

         Date     Open     High      Low    Close   Shares Traded   \
0  09-NOV-2015  7788.25  7937.75  7771.70  7915.20     218422388.0   
1  10-NOV-2015  7877.60  7885.10  7772.85  7783.35     170267413.0   
2  11-NOV-2015  7838.80  7847.95  7819.10  7825.00      22380435.0   
3  13-NOV-2015  7762.45  7775.10  7730.90  7762.25     165876819.0   
4  16-NOV-2015  7732.95  7838.85  7714.15  7806.60     154134885.0   

   Turnover (₹ Cr)  
0          9376.17  
1          7153.47  
2          1123.44  
3          7731.55  
4          6871.15  


In [ ]:
# Check for null values in each column
null_values = data.isnull().sum()
print(null_values)
data.fillna(method='ffill', inplace=True)
null_values = data.isnull().sum()
print(null_values)

Date               0
Open               0
High               0
Low                0
Close              0
Shares Traded      2
Turnover (₹ Cr)    2
dtype: int64
Date               0
Open               0
High               0
Low                0
Close              0
Shares Traded      0
Turnover (₹ Cr)    0
dtype: int64


<ipython-input-213-598fffa502c1>:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill', inplace=True)


In [ ]:
import pandas_ta as ta


In [ ]:
# Remove any extra spaces from column names
data.columns = data.columns.str.strip()

# Check the updated column names
print(data.columns)


Index(['Date', 'Open', 'High', 'Low', 'Close', 'Shares Traded',
       'Turnover (₹ Cr)'],
      dtype='object')


In [ ]:
print(data.columns)


Index(['Date', 'Open', 'High', 'Low', 'Close', 'Shares Traded',
       'Turnover (₹ Cr)'],
      dtype='object')


In [ ]:
# Check column names with their exact representation
print([repr(col) for col in data.columns])


["'Date'", "'Open'", "'High'", "'Low'", "'Close'", "'Shares Traded'", "'Turnover (₹ Cr)'"]


In [ ]:
import pandas_ta as ta

# Calculate Moving Averages (e.g., 5-day, 10-day, 20-day)
data['SMA_5'] = ta.sma(data['Close'], 5)
data['SMA_10'] = ta.sma(data['Close'], 10)
data['SMA_20'] = ta.sma(data['Close'], 20)

# Calculate RSI (Relative Strength Index) with a 14-day period
data['RSI_14'] = ta.rsi(data['Close'], 14)


# Calculate ATR (Average True Range)
data['ATR'] = ta.atr(data['High'], data['Low'], data['Close'], 14)

# Display the first few rows to check the results
print(data[['Date', 'Close', 'SMA_5', 'SMA_10', 'SMA_20', 'RSI_14',  'ATR']].head())


          Date    Close    SMA_5  SMA_10  SMA_20  RSI_14  ATR
0  09-NOV-2015  7915.20      NaN     NaN     NaN     NaN  NaN
1  10-NOV-2015  7783.35      NaN     NaN     NaN     NaN  NaN
2  11-NOV-2015  7825.00      NaN     NaN     NaN     NaN  NaN
3  13-NOV-2015  7762.25      NaN     NaN     NaN     NaN  NaN
4  16-NOV-2015  7806.60  7818.48     NaN     NaN     NaN  NaN


In [ ]:
print(data[['Date', 'Close', 'SMA_5', 'SMA_10', 'SMA_20', 'RSI_14',  'ATR']].head(30))


           Date    Close    SMA_5    SMA_10     SMA_20     RSI_14        ATR
0   09-NOV-2015  7915.20      NaN       NaN        NaN        NaN        NaN
1   10-NOV-2015  7783.35      NaN       NaN        NaN        NaN        NaN
2   11-NOV-2015  7825.00      NaN       NaN        NaN        NaN        NaN
3   13-NOV-2015  7762.25      NaN       NaN        NaN        NaN        NaN
4   16-NOV-2015  7806.60  7818.48       NaN        NaN        NaN        NaN
5   17-NOV-2015  7837.55  7802.95       NaN        NaN        NaN        NaN
6   18-NOV-2015  7731.80  7792.64       NaN        NaN        NaN        NaN
7   19-NOV-2015  7842.75  7796.19       NaN        NaN        NaN        NaN
8   20-NOV-2015  7856.55  7815.05       NaN        NaN        NaN        NaN
9   23-NOV-2015  7849.25  7823.58  7821.030        NaN        NaN        NaN
10  24-NOV-2015  7831.60  7822.39  7812.670        NaN        NaN        NaN
11  26-NOV-2015  7883.80  7852.79  7822.715        NaN        NaN        NaN

In [ ]:
# Define a function to calculate the Stochastic Oscillator %K and %D
def calculate_stochastic_oscillator(df, lookback_period=14, smoothing_period=3):
    # Calculate the rolling minimum and maximum over the lookback period
    df['Lowest_Low'] = df['Low'].rolling(window=lookback_period).min()  # Dynamic calculation of Lowest_Low
    df['Highest_High'] = df['High'].rolling(window=lookback_period).max()  # Dynamic calculation of Highest_High

    # Calculate %K
    df['%K'] = ((df['Close'] - df['Lowest_Low']) / (df['Highest_High'] - df['Lowest_Low'])) * 100

    # Calculate %D (3-period SMA of %K)
    df['%D'] = df['%K'].rolling(window=smoothing_period).mean()

    return df

# Apply the function to the data
data = calculate_stochastic_oscillator(data)

In [ ]:
print(data.head(30))

           Date     Open     High      Low    Close  Shares Traded  \
0   09-NOV-2015  7788.25  7937.75  7771.70  7915.20    218422388.0   
1   10-NOV-2015  7877.60  7885.10  7772.85  7783.35    170267413.0   
2   11-NOV-2015  7838.80  7847.95  7819.10  7825.00     22380435.0   
3   13-NOV-2015  7762.45  7775.10  7730.90  7762.25    165876819.0   
4   16-NOV-2015  7732.95  7838.85  7714.15  7806.60    154134885.0   
5   17-NOV-2015  7848.75  7860.45  7793.00  7837.55    149451211.0   
6   18-NOV-2015  7823.15  7843.40  7725.05  7731.80    148037721.0   
7   19-NOV-2015  7788.50  7854.90  7765.45  7842.75    136702518.0   
8   20-NOV-2015  7841.90  7906.95  7817.80  7856.55    156610433.0   
9   23-NOV-2015  7869.50  7877.50  7825.20  7849.25    130871603.0   
10  24-NOV-2015  7837.00  7870.35  7812.65  7831.60    134562905.0   
11  26-NOV-2015  7837.15  7897.10  7832.00  7883.80    226632193.0   
12  27-NOV-2015  7910.60  7959.30  7879.45  7942.70    154863047.0   
13  30-NOV-2015  793

In [ ]:
# Define a function to calculate the Stochastic Oscillator %K
def calculate_stochastic_oscillator(df, lookback_period=14, smoothing_period=3):
    # Calculate the rolling minimum and maximum over the lookback period
    df['Lowest_Low'] = df['Low'].rolling(window=lookback_period).min()
    df['Highest_High'] = df['High'].rolling(window=lookback_period).max()

    # Calculate %K
    df['%K'] = ((df['Close'] - df['Lowest_Low']) / (df['Highest_High'] - df['Lowest_Low'])) * 100

    # Calculate %D (3-period SMA of %K)
    df['%D'] = df['%K'].rolling(window=smoothing_period).mean()

    return df

# Apply the function to the data
data = calculate_stochastic_oscillator(data)

In [ ]:
print(data.head(30))

           Date     Open     High      Low    Close  Shares Traded  \
0   09-NOV-2015  7788.25  7937.75  7771.70  7915.20    218422388.0   
1   10-NOV-2015  7877.60  7885.10  7772.85  7783.35    170267413.0   
2   11-NOV-2015  7838.80  7847.95  7819.10  7825.00     22380435.0   
3   13-NOV-2015  7762.45  7775.10  7730.90  7762.25    165876819.0   
4   16-NOV-2015  7732.95  7838.85  7714.15  7806.60    154134885.0   
5   17-NOV-2015  7848.75  7860.45  7793.00  7837.55    149451211.0   
6   18-NOV-2015  7823.15  7843.40  7725.05  7731.80    148037721.0   
7   19-NOV-2015  7788.50  7854.90  7765.45  7842.75    136702518.0   
8   20-NOV-2015  7841.90  7906.95  7817.80  7856.55    156610433.0   
9   23-NOV-2015  7869.50  7877.50  7825.20  7849.25    130871603.0   
10  24-NOV-2015  7837.00  7870.35  7812.65  7831.60    134562905.0   
11  26-NOV-2015  7837.15  7897.10  7832.00  7883.80    226632193.0   
12  27-NOV-2015  7910.60  7959.30  7879.45  7942.70    154863047.0   
13  30-NOV-2015  793

In [ ]:
# Function to calculate Fibonacci Retracement levels
def calculate_fibonacci(df):
    # Calculate the Fibonacci retracement levels for each row
    df['Fibonacci_23.6'] = df['Low'] + 0.236 * (df['High'] - df['Low'])
    df['Fibonacci_38.2'] = df['Low'] + 0.382 * (df['High'] - df['Low'])
    df['Fibonacci_50.0'] = df['Low'] + 0.500 * (df['High'] - df['Low'])
    df['Fibonacci_61.8'] = df['Low'] + 0.618 * (df['High'] - df['Low'])
    df['Fibonacci_100'] = df['High']

    return df

# Apply the function to calculate Fibonacci retracement levels
data = calculate_fibonacci(data)
print(data.head(50))

           Date     Open     High      Low    Close  Shares Traded  \
0   09-NOV-2015  7788.25  7937.75  7771.70  7915.20    218422388.0   
1   10-NOV-2015  7877.60  7885.10  7772.85  7783.35    170267413.0   
2   11-NOV-2015  7838.80  7847.95  7819.10  7825.00     22380435.0   
3   13-NOV-2015  7762.45  7775.10  7730.90  7762.25    165876819.0   
4   16-NOV-2015  7732.95  7838.85  7714.15  7806.60    154134885.0   
5   17-NOV-2015  7848.75  7860.45  7793.00  7837.55    149451211.0   
6   18-NOV-2015  7823.15  7843.40  7725.05  7731.80    148037721.0   
7   19-NOV-2015  7788.50  7854.90  7765.45  7842.75    136702518.0   
8   20-NOV-2015  7841.90  7906.95  7817.80  7856.55    156610433.0   
9   23-NOV-2015  7869.50  7877.50  7825.20  7849.25    130871603.0   
10  24-NOV-2015  7837.00  7870.35  7812.65  7831.60    134562905.0   
11  26-NOV-2015  7837.15  7897.10  7832.00  7883.80    226632193.0   
12  27-NOV-2015  7910.60  7959.30  7879.45  7942.70    154863047.0   
13  30-NOV-2015  793

In [ ]:
# Sample df (with 'Date' and 'Close')
# df = pd.DataFrame({'Date': dates, 'Close': closing_prices})

# Calculate the MACD Line (12-period EMA minus 26-period EMA)
data['EMA_12'] = data['Close'].ewm(span=12, adjust=False).mean()  # 12-period EMA
data['EMA_26'] = data['Close'].ewm(span=26, adjust=False).mean()  # 26-period EMA

data['MACD'] = data['EMA_12'] - data['EMA_26']

# Calculate the Signal Line (9-period EMA of MACD Line)
data['Signal_Line'] = data['MACD'].ewm(span=9, adjust=False).mean()

# Calculate the MACD Histogram (MACD Line - Signal Line)
data['MACD_Histogram'] = data['MACD'] - data['Signal_Line']

# Now you have 'MACD', 'Signal_Line', and 'MACD_Histogram' columns
print(data.head(30))

           Date     Open     High      Low    Close  Shares Traded  \
0   09-NOV-2015  7788.25  7937.75  7771.70  7915.20    218422388.0   
1   10-NOV-2015  7877.60  7885.10  7772.85  7783.35    170267413.0   
2   11-NOV-2015  7838.80  7847.95  7819.10  7825.00     22380435.0   
3   13-NOV-2015  7762.45  7775.10  7730.90  7762.25    165876819.0   
4   16-NOV-2015  7732.95  7838.85  7714.15  7806.60    154134885.0   
5   17-NOV-2015  7848.75  7860.45  7793.00  7837.55    149451211.0   
6   18-NOV-2015  7823.15  7843.40  7725.05  7731.80    148037721.0   
7   19-NOV-2015  7788.50  7854.90  7765.45  7842.75    136702518.0   
8   20-NOV-2015  7841.90  7906.95  7817.80  7856.55    156610433.0   
9   23-NOV-2015  7869.50  7877.50  7825.20  7849.25    130871603.0   
10  24-NOV-2015  7837.00  7870.35  7812.65  7831.60    134562905.0   
11  26-NOV-2015  7837.15  7897.10  7832.00  7883.80    226632193.0   
12  27-NOV-2015  7910.60  7959.30  7879.45  7942.70    154863047.0   
13  30-NOV-2015  793

In [ ]:
# Convert 'Date' to datetime format
data['Date'] = pd.to_datetime(data['Date'], format='%d-%b-%Y')

# Calculate the Volume * Price for each row
data['Volume_Price'] = data['Close'] * data['Shares Traded']

# Calculate cumulative VWAP (VWAP = Sum of (Volume * Price) / Sum of Volume)
data['Cumulative_VWAP'] = data['Volume_Price'].cumsum() / data['Shares Traded'].cumsum()
print(data.head(30))

         Date     Open     High      Low    Close  Shares Traded  \
0  2015-11-09  7788.25  7937.75  7771.70  7915.20    218422388.0   
1  2015-11-10  7877.60  7885.10  7772.85  7783.35    170267413.0   
2  2015-11-11  7838.80  7847.95  7819.10  7825.00     22380435.0   
3  2015-11-13  7762.45  7775.10  7730.90  7762.25    165876819.0   
4  2015-11-16  7732.95  7838.85  7714.15  7806.60    154134885.0   
5  2015-11-17  7848.75  7860.45  7793.00  7837.55    149451211.0   
6  2015-11-18  7823.15  7843.40  7725.05  7731.80    148037721.0   
7  2015-11-19  7788.50  7854.90  7765.45  7842.75    136702518.0   
8  2015-11-20  7841.90  7906.95  7817.80  7856.55    156610433.0   
9  2015-11-23  7869.50  7877.50  7825.20  7849.25    130871603.0   
10 2015-11-24  7837.00  7870.35  7812.65  7831.60    134562905.0   
11 2015-11-26  7837.15  7897.10  7832.00  7883.80    226632193.0   
12 2015-11-27  7910.60  7959.30  7879.45  7942.70    154863047.0   
13 2015-11-30  7936.25  7966.00  7922.80  7935.2

In [ ]:
data = data.apply(lambda x: x.fillna(x.median()) if x.isnull().any() else x)
(print(data.head(30)))

         Date     Open     High      Low    Close  Shares Traded  \
0  2015-11-09  7788.25  7937.75  7771.70  7915.20    218422388.0   
1  2015-11-10  7877.60  7885.10  7772.85  7783.35    170267413.0   
2  2015-11-11  7838.80  7847.95  7819.10  7825.00     22380435.0   
3  2015-11-13  7762.45  7775.10  7730.90  7762.25    165876819.0   
4  2015-11-16  7732.95  7838.85  7714.15  7806.60    154134885.0   
5  2015-11-17  7848.75  7860.45  7793.00  7837.55    149451211.0   
6  2015-11-18  7823.15  7843.40  7725.05  7731.80    148037721.0   
7  2015-11-19  7788.50  7854.90  7765.45  7842.75    136702518.0   
8  2015-11-20  7841.90  7906.95  7817.80  7856.55    156610433.0   
9  2015-11-23  7869.50  7877.50  7825.20  7849.25    130871603.0   
10 2015-11-24  7837.00  7870.35  7812.65  7831.60    134562905.0   
11 2015-11-26  7837.15  7897.10  7832.00  7883.80    226632193.0   
12 2015-11-27  7910.60  7959.30  7879.45  7942.70    154863047.0   
13 2015-11-30  7936.25  7966.00  7922.80  7935.2

In [ ]:
data.shape

(2231, 28)

In [ ]:
columns_with_nan = data.isnull().any()
print(columns_with_nan)

Date               False
Open               False
High               False
Low                False
Close              False
Shares Traded      False
Turnover (₹ Cr)    False
SMA_5              False
SMA_10             False
SMA_20             False
RSI_14             False
ATR                False
Lowest_Low         False
Highest_High       False
%K                 False
%D                 False
Fibonacci_23.6     False
Fibonacci_38.2     False
Fibonacci_50.0     False
Fibonacci_61.8     False
Fibonacci_100      False
EMA_12             False
EMA_26             False
MACD               False
Signal_Line        False
MACD_Histogram     False
Volume_Price       False
Cumulative_VWAP    False
dtype: bool


In [ ]:
# Check if any row has NaN values
rows_with_nan = data.isnull().any(axis=1)
print(rows_with_nan)


0       False
1       False
2       False
3       False
4       False
        ...  
2226    False
2227    False
2228    False
2229    False
2230    False
Length: 2231, dtype: bool


In [ ]:
# from sklearn.preprocessing import MinMaxScaler

# Create an instance of MinMaxScaler
scaler = MinMaxScaler()

# Define the list of columns to scale
scaled_columns = [
    'Open', 'High', 'Low', 'Close', 'Shares Traded', 'Turnover (₹ Cr)',
    'SMA_5', 'SMA_10', 'SMA_20', 'EMA_12', 'EMA_26',"RSI_14","ATR",'%K','%D',"Lowest_Low","Highest_High","Fibonacci_23.6","Fibonacci_38.2","Fibonacci_50.0",
    'Fibonacci_61.8', 'Fibonacci_100','MACD', 'Signal_Line', 'MACD_Histogram',
    'Volume_Price', 'Cumulative_VWAP'
]

kyle = data.drop(columns=['Date'])
print(kyle.head())

# scaled_columns = [
#     'Open', 'High', 'Low', 'Close'
# ]

# Check if all specified columns are present in the DataFrame
missing_columns = [col for col in scaled_columns if col not in data.columns]
if missing_columns:
    print(f"Warning: Missing columns - {missing_columns}")
else:
    # Scale the columns only if all are present
    data[scaled_columns] = scaler.fit_transform(data[scaled_columns])

# Display the scaled DataFrame (optional)
print(data.head())
df = data.drop(columns=["Date"])


       Open      High       Low     Close  Shares Traded  Turnover (₹ Cr)  \
0  0.039772  0.046954  0.048945  0.049082       0.112433         0.092173   
1  0.044420  0.044218  0.049005  0.042231       0.085605         0.068268   
2  0.042401  0.042288  0.051398  0.044395       0.003214         0.003415   
3  0.038430  0.038502  0.046834  0.041134       0.083159         0.074485   
4  0.036895  0.041815  0.045968  0.043439       0.076617         0.065231   

      SMA_5    SMA_10    SMA_20    RSI_14       ATR  Lowest_Low  Highest_High  \
0  0.250857  0.250675  0.249845  0.596408  0.191208    0.255897      0.250884   
1  0.250857  0.250675  0.249845  0.596408  0.191208    0.255897      0.250884   
2  0.250857  0.250675  0.249845  0.596408  0.191208    0.255897      0.250884   
3  0.250857  0.250675  0.249845  0.596408  0.191208    0.255897      0.250884   
4  0.041789  0.250675  0.249845  0.596408  0.191208    0.255897      0.250884   

         %K        %D  Fibonacci_23.6  Fibonacci_3

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
print(data.head())

        Date      Open      High       Low     Close  Shares Traded  \
0 2015-11-09  0.039772  0.046954  0.048945  0.049082       0.112433   
1 2015-11-10  0.044420  0.044218  0.049005  0.042231       0.085605   
2 2015-11-11  0.042401  0.042288  0.051398  0.044395       0.003214   
3 2015-11-13  0.038430  0.038502  0.046834  0.041134       0.083159   
4 2015-11-16  0.036895  0.041815  0.045968  0.043439       0.076617   

   Turnover (₹ Cr)     SMA_5    SMA_10    SMA_20    RSI_14       ATR  \
0         0.092173  0.250857  0.250675  0.249845  0.596408  0.191208   
1         0.068268  0.250857  0.250675  0.249845  0.596408  0.191208   
2         0.003415  0.250857  0.250675  0.249845  0.596408  0.191208   
3         0.074485  0.250857  0.250675  0.249845  0.596408  0.191208   
4         0.065231  0.041789  0.250675  0.249845  0.596408  0.191208   

   Lowest_Low  Highest_High        %K        %D  Fibonacci_23.6  \
0    0.255897      0.250884  0.686708  0.682498        0.047774   
1    0

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

def create_lstm_dataset(d, time_steps=1):
    X = []
    y = []

    target_columns = ['Open', 'High', 'Low', 'Close']  # Define the target columns

    # Create a scaler for the target columns only
    scaler = MinMaxScaler()
    d[target_columns] = scaler.fit_transform(d[target_columns])

    # Loop through the data to create X and y
    for i in range(time_steps, len(d)):
        # Get the features (scaled data for time_steps)
        X.append(d[i - time_steps:i])

        # Get the target values ('Open', 'High', 'Low', 'Close') for the next day
        target_values = d.iloc[i][target_columns].values
        y.append(target_values)

    # Convert lists to numpy arrays for model input
    X = np.array(X)
    y = np.array(y)

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    return X_train, X_test, y_train, y_test, scaler

X_train, X_test, y_train, y_test, scaler = create_lstm_dataset(df, time_steps=10)


X_train, X_test, y_train, y_test, scaler = create_lstm_dataset(df, time_steps=10)

# Build the LSTM model
model = Sequential()

# Add LSTM layer
model.add(LSTM(units=50, return_sequences=False, input_shape=(X_train.shape[1], X_train.shape[2])))

# Add dropout for regularization
model.add(Dropout(0.2))

# Output layer for multiple outputs (4 values: 'Open', 'High', 'Low', 'Close')
model.add(Dense(units=4))  # Output layer should have 4 units, one for each target column

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))



Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


56/56 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - loss: 0.0537 - val_loss: 0.0110
Epoch 2/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0066 - val_loss: 0.0066
Epoch 3/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0040 - val_loss: 0.0083
Epoch 4/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0031 - val_loss: 0.0039
Epoch 5/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0026 - val_loss: 0.0044
Epoch 6/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0023 - val_loss: 0.0038
Epoch 7/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0020 - val_loss: 0.0046
Epoch 8/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0018 - val_loss: 0.0023
Epoch 9/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0016 - val_loss: 0.0049
Epoch 10/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0016 - val_loss: 0.0016
Epoch 11/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0015 - val_loss: 0.0018
Epoch 12/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0014 - val_loss: 0.0

In [ ]:
print("Input shape:", model.input_shape)

Input shape: (None, 10, 27)


In [ ]:
# Evaluate the model on the test data
test_loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}')

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.5788e-04 
Test Loss: 0.00043490983080118895


In [ ]:
# Make predictions on the test set
y_pred = model.predict(X_test)

# Print a sample of predictions vs actual values
print("Predictions: ", y_pred[:5])
print("Actual values: ", y_test[:5])


14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step
Predictions:  [[0.5987867  0.5867473  0.5870717  0.58556724]
 [0.60236365 0.589811   0.5886166  0.5864049 ]
 [0.59491533 0.5799148  0.58071434 0.57929206]
 [0.58674514 0.57067573 0.56873685 0.5667778 ]
 [0.58419365 0.56754124 0.56343997 0.563594  ]]
Actual values:  [[0.58052183 0.58031299 0.58227688 0.57923821]
 [0.57580912 0.57508256 0.5702462  0.56747699]
 [0.56456571 0.56386558 0.55200097 0.5525332 ]
 [0.54712712 0.55474026 0.54744743 0.55485063]
 [0.55698428 0.55611997 0.55427775 0.5555365 ]]


In [ ]:
y_pred_rescaled = scaler.inverse_transform(y_pred)
y_test_rescaled = scaler.inverse_transform(y_test)

print("Predicted values (rescaled):", y_pred_rescaled[:5])
print("Actual values (rescaled):", y_test_rescaled[:5])

ValueError: operands could not be broadcast together with shapes (445,4) (27,) (445,4) 

In [ ]:
# Reshape your input data to match the expected 3D shape
# The model expects data in the shape (samples, time steps, features)
# Your training data has 10 time steps and likely 27 features (based on the error message)
# You need to provide input data that matches this shape

# Example: If you want to predict based on the last 10 days of data
# and your original data has 27 features, you'll need to reshape it accordingly

# Extract the last 10 rows from your dataset, or whatever data
# you want to use for prediction and extract all features
# used in training
# Ensure that last_10_days_data has the same columns as df
last_10_days_data = kyle[-10:]  # Extract last 10 days from 'df'

# Convert to a numpy array
last_10_days_data = last_10_days_data.values

# Instead of using feature_columns directly for indexing,
# Select the columns from 'last_10_days_data' using integer indices
input_data = last_10_days_data[:, :len(feature_columns)] #Select only the columns used during training

# Reshape to 3D
input_data = input_data.reshape(1, input_data.shape[0], input_data.shape[1])
print("input data givng:- ",input_data)
# Make the prediction
prediction = model.predict(input_data)
prediction = prediction.reshape(1, -1)

# Inverse transform the prediction to get the values in the original scale
prediction_rescaled = scaler.inverse_transform(prediction)

print("Predicted values (rescaled):", prediction_rescaled)

input data givng:-  [[[0.89611487 0.90725271 0.89565654 0.90247565 0.1459377  0.29393427
   0.9111876  0.93479437 0.96682487 0.29528065 0.44067426 0.93716754
   0.94516149 0.22477213 0.09697423 0.89830922 0.89995494 0.90133018
   0.90272611 0.90725271 0.93940196 0.96822054 0.51848584 0.54393388
   0.3904599  0.30339385 0.9944949 ]
  [0.90015917 0.90683178 0.89596442 0.90911098 0.17573372 0.37398779
   0.91113243 0.93164606 0.96316028 0.35535759 0.45163283 0.93716754
   0.94400511 0.34304347 0.19499497 0.89844577 0.89998519 0.90127437
   0.9025842  0.90683178 0.93799297 0.96651191 0.5218354  0.53060603
   0.43491318 0.36483981 0.99524505]
  [0.90237508 0.90754372 0.90457735 0.90256398 0.14963368 0.32547463
   0.91013782 0.92828476 0.95919018 0.32002938 0.43874168 0.93716754
   0.94400511 0.23076897 0.24483469 0.90520514 0.90559462 0.90595056
   0.90632557 0.90754372 0.9357624  0.96441828 0.51946621 0.51939676
   0.45682187 0.31079748 0.99588131]
  [0.90125152 0.90100893 0.89760732 0.895

ValueError: operands could not be broadcast together with shapes (1,4) (27,) (1,4) 

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


ValueError: math domain error